### Installing required packages

In [ ]:
!pip uninstall transformers -y

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install byaldi together pdf2image

In [ ]:
!sudo apt-get install -y poppler-utils

In [ ]:
import os
#!huggingface-cli login

from huggingface_hub import login
login(os.environ.get("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Initialize the ColPali Model

In [ ]:
import os
from pathlib import Path
from byaldi import RAGMultiModalModel

# Initialize RAGMultiModalModel
model = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v0.1")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

### **Preparing data**

In [ ]:
import pandas as pd

# Read the file
file_path = r'/content/charts_evaluations.txt'

# Load the data into a DataFrame, assuming each line contains a question-answer pair separated by a comma
df = pd.read_csv(file_path, delimiter=',', names=['Question', 'Answer'])

# Display the DataFrame
print(df['Answer'][5])

 حوالي 4%.


### Lets create our index that will store the embeddings for the page images.

Caution: This cell below takes ~5 mins to index the whole PDF!

In [ ]:
# Use ColQwen2 to index and store the presentation
index_name = "test_index"
model.index(input_path=Path("/content/Econ_3.pdf"),
    index_name=index_name,
    store_collection_with_index=True, # Stores base64 images along with the vectors
    overwrite=True
)

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of document 0 to index.
Added page 26 of document 0 to index.
Added page 27 of docu

{0: '/content/Econ_3.pdf'}

### Indexing takes a lot of time

### This concludes the indexing of the PDF phase - everything below happens at query time.

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/colpali_arch.png?raw=1" width="700">

### Let's query our indexed document.


In [ ]:
import pandas as pd
import time
from together import Together

# Initialize the Together client with your API key
api_key = os.environ.get("TOGETHER_API_KEY")
client = Together(api_key=api_key)

In [ ]:
df.head()

,Question,Answer
0,ما هي الدولة أو المنطقة التي شهدت أقل نسبة نمو...,الاقتصادات المتقدمة شهدت أقل نسبة نمو للناتج ...
1,كيف كان أداء الصين في عام 2021 مقارنةً بعام 20...,شهدت الصين تحسناً ملحوظاً في نسبة نمو الناتج ...
2,ما الفرق بين معدلات النمو في الاقتصادات المتقد...,في عام 2019، كانت نسبة النمو في الاقتصادات ال...
3,ما هي نسبة التغير في أسعار المستهلكين في الاقت...,حوالي 1%.
4,ما هي المجموعة التي شهدت أعلى نسبة تغير في أسع...,دول الشرق الأوسط وشمال أفريقيا، حيث بلغت النس...


In [ ]:
def process_queries(df):
    answers = []
    for index, row in df.iterrows():
        query = row['Question']
        try:
            # Perform search to get the most relevant page
            results = model.search(query, k=1)
            if results:
                returned_page_base64 = results[0].base64
                # Prepare the messages with an instruction in Arabic
                messages = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "الرجاء الإجابة باللغة العربية وبشكل دقيق ومختصر."},
                            {"type": "text", "text": query},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{returned_page_base64}",
                                },
                            },
                        ],
                    }
                ]
                # Call the Together API to get the answer
                response = client.chat.completions.create(
                    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
                    messages=messages,
                    max_tokens=300,
                )
                # Extract the answer from the response
                answer = response.choices[0].message.content
                answers.append(answer)
            else:
                answers.append(None)
        except Exception as e:
            print(f"Error processing query at index {index}: {e}")
            answers.append(None)
            # Sleep to handle any rate limiting
            time.sleep(1)
    # Add the answers as a new column in the DataFrame
    df['Responses'] = answers
    return df


In [ ]:
x = process_queries(df)
x.save('results.csv')

,Question,Answer,Responses
0,ما هي الدولة أو المنطقة التي شهدت أقل نسبة نمو...,الاقتصادات المتقدمة شهدت أقل نسبة نمو للناتج ...,للإجابة على هذا السؤال، نحتاج إلى العودة إلى ا...
1,كيف كان أداء الصين في عام 2021 مقارنةً بعام 20...,شهدت الصين تحسناً ملحوظاً في نسبة نمو الناتج ...,**تجاوز الناتج المحلي الإجمالي الصيني في 2021 ...
2,ما الفرق بين معدلات النمو في الاقتصادات المتقد...,في عام 2019، كانت نسبة النمو في الاقتصادات ال...,في عام 2019، كانت معدلات النمو في الاقتصادات ا...
3,ما هي نسبة التغير في أسعار المستهلكين في الاقت...,حوالي 1%.,النسبة المئوية للتغير في أسعار المستهلكين في ا...
4,ما هي المجموعة التي شهدت أعلى نسبة تغير في أسع...,دول الشرق الأوسط وشمال أفريقيا، حيث بلغت النس...,المجموعة التي شهدت أعلى نسبة تغير في أسعار الم...
5,كيف كانت نسبة التغير المتوقعة في أسعار المستهل...,حوالي 4%.,تم توقع ارتفاع أسعار المستهلكين في الأسواق الن...
6,ما هي نسبة البطالة في منطقة اليورو في عام 2018؟,حوالي 8%.,نسبة البطالة في منطقة اليورو في عام 2018 هي 7.9%
7,ما هي نسبة البطالة في الاقتصادات المتقدمة في ع...,حوالي 5%.,نسبة البطالة في الاقتصادات المتقدمة في عام 201...
8,كيف كانت نسبة البطالة المتوقعة لمنطقة اليورو ف...,حوالي 8%.,**8%**\n\nهذا الفصل يحتوي على تقديرات الإنتاجي...
9,ما الفرق بين نسبة البطالة في منطقة اليورو والا...,كانت نسبة البطالة في منطقة اليورو حوالي 8%، ب...,في عام 2020، كانت نسبة البطالة في منطقة اليورو...


In [ ]:
x = process_queries(df)


In [ ]:
x.to_csv('results.csv', index=False,encoding = 'utf-8-sig')

### Evaluation

In [ ]:
!wget -O wiki_cbow_300.zip "https://ia903107.us.archive.org/0/items/aravec2.0/wiki_cbow_300.zip"

!unzip wiki_cbow_300.zip -d aravec_model

--2024-11-02 10:40:38--  https://ia903107.us.archive.org/0/items/aravec2.0/wiki_cbow_300.zip
Resolving ia903107.us.archive.org (ia903107.us.archive.org)... 207.241.232.147
Connecting to ia903107.us.archive.org (ia903107.us.archive.org)|207.241.232.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364888893 (348M) [application/zip]
Saving to: ‘wiki_cbow_300.zip’

wiki_cbow_300.zip   100%[===================>] 347.98M  2.34MB/s    in 61s     

2024-11-02 10:41:40 (5.74 MB/s) - ‘wiki_cbow_300.zip’ saved [364888893/364888893]

Archive:  wiki_cbow_300.zip
  inflating: aravec_model/wikipedia_cbow_300  
  inflating: aravec_model/wikipedia_cbow_300.trainables.syn1neg.npy  
  inflating: aravec_model/wikipedia_cbow_300.wv.vectors.npy  


In [ ]:
import gensim

# Load the model
model_path = '/content/aravec_model/wikipedia_cbow_300'
aravec_model = gensim.models.Word2Vec.load(model_path)


In [ ]:
import pandas as pd
import re
import numpy as np

# Function to normalize Arabic text
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)  # Normalize Alef
    text = re.sub("ى", "ي", text)       # Normalize Ya
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[ًٌٍَُِّْـ]", "", text)  # Remove diacritics and tatweel
    return text

# Function to remove stop words
def remove_stopwords(tokens):
    arabic_stopwords = set([
        'في', 'من', 'على', 'ما', 'عن', 'إلى', 'و', 'لا', 'نعم', 'هل',
        'ثم', 'هنا', 'هناك', 'كل', 'بعض', 'ذلك', 'هذا', 'هذه', 'هو', 'هي',
        # Add more stop words as needed
    ])
    return [word for word in tokens if word not in arabic_stopwords]

# Function to preprocess Arabic text
def preprocess_text(text):
    text = normalize_arabic(text)
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)      # Remove digits
    text = text.strip()
    tokens = text.split()
    tokens = remove_stopwords(tokens)
    return tokens  # Return a list of tokens


# Function to get the embedding of a text
def get_text_embedding(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])
    if embeddings:
        # Compute the mean of the embeddings
        mean_embedding = np.mean(embeddings, axis=0)
    else:
        # If no tokens are in the vocabulary, return a zero vector
        mean_embedding = np.zeros(model.vector_size)
    return mean_embedding



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity_aravec(df, threshold=0.5):
    similarities = []

    for index, row in df.iterrows():
        # Preprocess the texts
        tokens_response = preprocess_text(str(row['Responses']))
        tokens_answer = preprocess_text(str(row['Answer']))

        # Get embeddings
        embedding_response = get_text_embedding(tokens_response, aravec_model)
        embedding_answer = get_text_embedding(tokens_answer, aravec_model)

        # Compute cosine similarity
        sim = cosine_similarity([embedding_response], [embedding_answer])[0][0]
        similarities.append(sim)

    # Add similarities to the DataFrame
    df['similarity'] = similarities
    # Apply threshold to get labels
    df['label'] = df['similarity'].apply(lambda x: 1 if x >= threshold else 0)

    return df


In [ ]:
# Assuming 'df' is your DataFrame with 'responses' and 'Answers' columns
compute_similarity_aravec(df, threshold=0.2)

df

,Question,Answer,Responses,similarity,label
0,ما هي الدولة أو المنطقة التي شهدت أقل نسبة نمو...,الاقتصادات المتقدمة شهدت أقل نسبة نمو للناتج ...,يُظهر الرسم البياني رقم 6-2: الميزانية التقدير...,0.750350,1
1,كيف كان أداء الصين في عام 2021 مقارنةً بعام 20...,شهدت الصين تحسناً ملحوظاً في نسبة نمو الناتج ...,استنادًا إلى تقرير الإنجازات الاقتصادية الصيني...,0.687817,1
2,ما الفرق بين معدلات النمو في الاقتصادات المتقد...,في عام 2019، كانت نسبة النمو في الاقتصادات ال...,الفرق بين معدلات النمو في الاقتصادات المتقدمة ...,0.671083,1
3,ما هي نسبة التغير في أسعار المستهلكين في الاقت...,حوالي 1%.,نسبة التغير في أسعار المستهلكين في الاقتصادات ...,0.170475,0
4,ما هي المجموعة التي شهدت أعلى نسبة تغير في أسع...,دول الشرق الأوسط وشمال أفريقيا، حيث بلغت النس...,**المنتجات الغذائية والمشروبات** هي المجموعة ا...,0.440773,1
5,كيف كانت نسبة التغير المتوقعة في أسعار المستهل...,حوالي 4%.,**2.3%**,0.000000,0
6,ما هي نسبة البطالة في منطقة اليورو في عام 2018؟,حوالي 8%.,7.6%,0.000000,0
7,ما هي نسبة البطالة في الاقتصادات المتقدمة في ع...,حوالي 5%.,نسبة البطالة في الاقتصادات المتقدمة في عام 201...,0.285101,1
8,كيف كانت نسبة البطالة المتوقعة لمنطقة اليورو ف...,حوالي 8%.,نسبة البطالة المتوقعة لمنطقة اليورو في عام 202...,0.240742,1
9,ما الفرق بين نسبة البطالة في منطقة اليورو والا...,كانت نسبة البطالة في منطقة اليورو حوالي 8%، ب...,الفرق بين نسبة البطالة في منطقة اليورو والاقتص...,0.623268,1


In [ ]:
df['label'].value_counts()

,count
label,
1,33
0,8


In [ ]:
33/41

0.8048780487804879

In [ ]:
def print_relevant_pairs(df):
    # Filter the DataFrame for rows where 'label' == 1
    relevant_pairs = df[df['label'] == 1]

    # Iterate over the filtered rows
    for index, row in relevant_pairs.iterrows():
        print(f"Pair {index + 1}:")
        print(f"Response: {row['Responses']}")
        print(f"Answer: {row['Answer']}")
        print(f"Similarity Score: {row['similarity']:.4f}")
        print("-" * 50)  # Separator line


In [ ]:
print_relevant_pairs(df)

Pair 1:
Response: يُظهر الرسم البياني رقم 6-2: الميزانية التقديرية والعملية للعام المالي 1442/1441هـ (2020م) أن أقل نسبة نمو للناتج المحلي الإجمالي في عام 2020 كانت في الفاتح / العجز، حيث بلغت -1.7% (في المليون ريال).
Answer:  الاقتصادات المتقدمة شهدت أقل نسبة نمو للناتج المحلي الإجمالي في عام 2020، حيث وصلت النسبة إلى حوالي -5%.
Similarity Score: 0.7503
--------------------------------------------------
Pair 2:
Response: استنادًا إلى تقرير الإنجازات الاقتصادية الصينية في عام 2021، حققت الصين نموًا بنسبة 8.1٪ مقارنةً بـ 2.3٪ في عام 2020، مما يدل على تحسن كبير في الأداء الاقتصادي.
Answer:  شهدت الصين تحسناً ملحوظاً في نسبة نمو الناتج المحلي الإجمالي في عام 2021 مقارنةً بعام 2020، حيث ارتفعت النسبة من حوالي 2% إلى ما يقارب 8%.
Similarity Score: 0.6878
--------------------------------------------------
Pair 3:
Response: الفرق بين معدلات النمو في الاقتصادات المتقدمة ودول الأسواق الناشئة والنامية في عام 2019 هو 1.8%. 

حيث بلغت معدلات النمو في الاقتصاد المتقدم 2.1%، بينما بلغت معدلات النمو 

In [ ]:
# Save the DataFrame to a CSV file with utf-8-sig encoding
df.to_csv('similarity_results_aravec_wiki.csv', index=False, encoding='utf-8-sig')